In [4]:
import os, sys, numpy, pylab,importlib, pickle, gzip, json, re
from covid19_stats.engine import core, gis, viz
from covid19_stats import COVID19Database
from matplotlib.patches import Polygon
from itertools import chain
from rapidfuzz.fuzz import ratio
from nprstuff.core import autocrop_image
%matplotlib inline

The `COVID19Database` singleton object, and the location of the `core` subdirectory for SPHINX docstring documentation.

In [6]:
cdat = COVID19Database( )
#static_core_dir = '../docsrc/source/_static/core'
#assert( os.path.isdir( static_core_dir ))

In [7]:
cdat.data_msas_2019().keys()

{'carsoncity': {'prefix': 'carsoncity',
  'region name': 'Carson City Metro Area',
  'fips': {'32510'},
  'population': 55916},
 'wallawalla': {'prefix': 'wallawalla',
  'region name': 'Walla Walla Metro Area',
  'fips': {'53071'},
  'population': 60760},
 'enid': {'prefix': 'enid',
  'region name': 'Enid Metro Area',
  'fips': {'40047'},
  'population': 61056},
 'lewiston_id-wa': {'prefix': 'lewiston_id-wa',
  'region name': 'Lewiston ID-WA Metro Area',
  'fips': {'16069', '53003'},
  'population': 62990},
 'grandisland': {'prefix': 'grandisland',
  'region name': 'Grand Island Metro Area',
  'fips': {'31079', '31093', '31121'},
  'population': 75553},
 'danville': {'prefix': 'danville',
  'region name': 'Danville Metro Area',
  'fips': {'17183'},
  'population': 75758},
 'casper': {'prefix': 'casper',
  'region name': 'Casper Metro Area',
  'fips': {'56025'},
  'population': 79858},
 'hinesville': {'prefix': 'hinesville',
  'region name': 'Hinesville Metro Area',
  'fips': {'13179', 

Gets low-level geographical data that I forgot from a few months ago!

In [3]:
df_fips = cdat.fips_dataframe_2019( )

In [49]:
df_sub = df_fips[ df_fips.state_or_territory == 'Virginia' ]
county_names_dict = dict(map(lambda county: (re.sub('city', '',
                                                    re.sub( 'county', '', county.lower())).strip(), county),
                                 sorted(df_sub.county )))
ratio('charlottesville', 'charlotte')

75.0

In [58]:
def get_closest_region_match(
    df_fips, county_name, state_name,
    minimum_partial = 0.9, county_names_dict = { } ):
    df_sub = df_fips[ df_fips.state_or_territory == state_name ]
    if df_sub.shape[0] == 0:
        raise ValueError("Error, no state or territory named %s." %
                         state_name )
    if len( county_names_dict ) == 0:
        county_names_dict = dict(map(lambda county: (
            re.sub('city', '', re.sub( 'county', '', county.lower())).strip(), county),
                                     sorted(df_sub.county )))
    best_match = max(county_names_dict, key = lambda clower: ratio(
        clower, county_name.lower( ) ) )
    best_match_score = ratio(
        county_name.lower(), best_match )
    if best_match_score < minimum_partial:
        return None
    return county_names_dict[ best_match ]
#
##
def get_closest_region_match_dict(
    df_fips, county_names, state_name, minimum_partial = 0.9 ):
    df_sub = df_fips[ df_fips.state_or_territory == state_name ]
    if df_sub.shape[1] == 0:
        raise ValueError("Error, no state or territory named %s." %
                         state_name )
    county_names_dict = dict(map(lambda county: (
        re.sub('city', '', re.sub( 'county', '', county.lower())).strip(), county),
                                 sorted(df_sub.county )))
    def _get_closest_match( county_name ):
        best_match = get_closest_region_match(
            df_sub, county_name, state_name,
            minimum_partial = minimum_partial,
            county_names_dict = county_names_dict )
        if best_match is None: return None
        return (county_name, best_match )

    county_match_dict = dict(map(_get_closest_match, county_names))
    len_vals = len( county_match_dict.values( ) )
    len_vals_set = len( set( county_match_dict.values( ) ) )
    if len_vals != len_vals_set:
        raise ValueError("Error, this cannot be right. Final list of candidate counties is %s." % (
            sorted( county_match_dict.values( )  ) ) )
    return county_match_dict

In [60]:
get_closest_region_match_dict(
    df_fips,
    ['Charlottesville','Albemarle','Nelson','Greene','Louisa','Fluvanna'], 'Virginia')

{'Charlottesville': 'Charlottesville city',
 'Albemarle': 'Albemarle County',
 'Nelson': 'Nelson County',
 'Greene': 'Greene County',
 'Louisa': 'Louisa County',
 'Fluvanna': 'Fluvanna County'}

In [3]:
data_brhd_pre = json.load(open('brhd.json', 'r'))
data_brhd = {
        'prefix' : data_brhd_pre[ 'prefix' ],
        'region name' : data_brhd_pre[ 'region name' ],
        'fips' : set( data_brhd_pre[ 'fips' ] ),
        'population' : data_brhd_pre[ 'population' ] }

Now get latest incident data for `BRHD` geographical region

In [4]:
inc_data = core.get_incident_data( data_brhd)

In [9]:
inc_data.keys(), inc_data['last day']

(dict_keys(['df', 'bbox', 'boundaries', 'df_7day', 'df_1day', 'last day', 'prefix', 'region name', 'fips', 'population']),
 580)

In [2]:
%%HTML
<video width="100%" controls>
    <source src="covid19_7day_brhd_LATEST.mp4" type="video/mp4">
</video>

In [3]:
%%HTML
<video width="100%" controls>
    <source src="covid19_brhd_LATEST.mp4" type="video/mp4">
</video>